# Working with volatility3 netscan output from a dc

Get some of the needed modules

In [1]:
import os
import codecs
import pandas as pd

Setup some paths to the files

In [3]:
PATH_TO_FOLDER = '/home/ubuntu/jupyter/cases/stolen_szechuan_sauce/'
CSV_FILE = 'dc-netscan.csv'
PATH_TO_CSV = os.path.join(PATH_TO_FOLDER, CSV_FILE)
print(PATH_TO_CSV)

/home/ubuntu/jupyter/cases/stolen_szechuan_sauce/dc-netscan.csv


Read the file into pandas

In [4]:
df = None
with codecs.open(PATH_TO_CSV, 'r', encoding='utf-8', errors='replace') as fh:
  df = pd.read_csv(fh, error_bad_lines=False)

print(df.shape)

(19685, 11)


In [5]:
df.head()

,TreeDepth,Offset,Proto,LocalAddr,LocalPort,ForeignAddr,ForeignPort,State,PID,Owner,Created
0,0,0x1600730,UDPv4,0.0.0.0,0,*,0,NaN,1368,dns.exe,2020-09-19 01:22:57.000000
1,0,0x1600730,UDPv6,::,0,*,0,NaN,1368,dns.exe,2020-09-19 01:22:57.000000
2,0,0x1600ec0,UDPv4,0.0.0.0,0,*,0,NaN,1368,dns.exe,2020-09-19 01:22:57.000000
3,0,0x1600ec0,UDPv6,::,0,*,0,NaN,1368,dns.exe,2020-09-19 01:22:57.000000
4,0,0x1601790,UDPv4,0.0.0.0,0,*,0,NaN,1368,dns.exe,2020-09-19 01:22:57.000000


Looks like we have workable data. We have 19,685 rows and 11 columns. Let's try to find the unique Foreign Addresses.

In [6]:
df["ForeignAddr"].value_counts()

*                            19511
0.0.0.0                         79
::                              44
fe80::2dcf:e660:be73:d220       28
::1                             21
203.78.103.109                   2
Name: ForeignAddr, dtype: int64

203.78.103.109 seems to stand out as it is not a link local address, a * or 0.0.0.0. Let's try to take a look at those lines.

In [7]:
df[df["ForeignAddr"] == "203.78.103.109"]

,TreeDepth,Offset,Proto,LocalAddr,LocalPort,ForeignAddr,ForeignPort,State,PID,Owner,Created
9209,0,0x20fc7590,TCPv4,10.42.85.10,62613,203.78.103.109,443,ESTABLISHED,3644,coreupdater.ex,NaN
18121,0,0x60182590,TCPv4,10.42.85.10,62613,203.78.103.109,443,ESTABLISHED,3644,coreupdater.ex,NaN


Looks like here we have a connection from the dc to the address 203.78.103.109 from the process coreupdater.ex. We have identified this process as suspcious before. 